## Connects to GBQ

In [ ]:
#connect to GBQ instance

# Do our imports for the code
import random
from google.cloud import bigquery
from google.oauth2 import service_account


# These first two values will be different on your machine. 
service_path = "C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\"
service_file = 'ada-Wedge-b7fc47715ad0.json' # this is your authentication information  
gbq_proj_id = 'ada-wedge'  # change this to your project_id
gbq_dataset_id = 'wedge2' # and change this to your data set ID

private_key =service_path + service_file

# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Builds List of Owners

In [ ]:
#builds list of owners from each table in database using below query


query = (
    "SELECT DISTINCT card_no "
    "FROM `ada-wedge.wedge.transArchive*` "
    "WHERE card_no != 3"
)


query_job = client.query(
    query,
    location="US",
)


results = client.query(query) 
#print(list(results))


#Puts each entry from results into seperate row
new_list = []
for entry in results :
    new_list.append(entry)
    

    
#uses enumerate, go through each row
#and pull out the first number of each entry which is the unique client number and put it in a list called owner_list
owner_list = []


for counter,entry in enumerate (new_list):
    owner_list.append(new_list[counter][0]) 

  
    

## Takes a sample of owners

In [ ]:
sample = random.choices(owner_list, k=500) #makes a random sample of 500 owners (about 291 MB)
#print(sample[0:3])

## Selects all info for each owner in sample

In [ ]:
##Selects every variable from the wedge database using the below query

query = ("""
    Select *
    FROM `ada-wedge.wedge2.transArchive*` 
    WHERE card_no in (
    
""")

#builds the query each time by adding the next owner in the random sample to it.
for idx, owner in enumerate(sample): 
    new_owner = (str(owner)[:-2]) #removes the .0 and turns it into a string
    if idx < len(sample)-1: #if the index is not the last one in the sample, then it adds a comma after it
        query += new_owner + ','
    else: #otherwise it ends the query with a )
        query += new_owner + ')'

    
    

#runs the query job

query_job = client.query(
    query,
    location="US",
)


sample_client_info = client.query(query) 



## Writes Owner Info into a file

In [ ]:
#this is the first row of headers 
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

In [ ]:
#writes the query to a file
with open('Sample_Owners.txt','a') as output_file:
    output_file.write(",".join(headers) + "\n") #adds headers as first line
    #turn each item into a string for each line in the sample_client_info (aka results) and save it as a row
    #write the row into the text file separating each item in the row by , and adding a carriage return after it
    for line in sample_client_info:
        row = [str(item) for item in line]
        output_file.write(",".join(row)+'\n')
        
    






